In [1]:
import numpy as np
import pandas as pd 

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

import os
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
os.path.join(PROJECT_ROOT_DIR)
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
CLEAREDDATA_PATH = os.path.join(PROJECT_ROOT_DIR, "cleareddata")
os.makedirs(IMAGES_PATH, exist_ok=True)
os.makedirs(CLEAREDDATA_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()

from pipe_classes import *

./data/scotch_review.csv
./data/nyc_benchmarking_disclosure_2017_consumption_data.xlsx


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:46: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/opt/conda/lib/python3.7/site-packages/tqdm/std.py:654: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
data = pd.read_excel(PROJECT_ROOT_DIR+'/data/nyc_benchmarking_disclosure_2017_consumption_data.xlsx','Information and Metrics')

In [3]:
data = data.replace({'Not Available': np.nan})

In [4]:
data.select_dtypes(exclude=['number']).keys()

Index(['Property Name', 'Parent Property Id', 'Parent Property Name',
       'NYC Borough, Block and Lot (BBL) self-reported',
       'NYC Building Identification Number (BIN)', 'Address 1 (self-reported)',
       'Address 2 (self-reported)', 'Postal Code', 'Street Number',
       'Street Name', 'Borough', 'DOF Gross Floor Area (ft²)',
       'Primary Property Type - Self Selected',
       'List of All Property Use Types at Property',
       'Largest Property Use Type', '2nd Largest Property Use Type',
       '3rd Largest Property Use Type', 'Metered Areas (Energy)',
       'Metered Areas  (Water)', 'Annual Maximum Demand (MM/YYYY)',
       'Water Required?', 'Generation Date',
       'DOF Benchmarking Submission Status'],
      dtype='object')

In [5]:
data.select_dtypes('number').keys().size

37

### First exercise: Ridge and SVR without PCA or collinear features remover and without the sqrt-log feature adder

In [6]:
from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

reg_target="ENERGY STAR Score"
spec_cols=['Water Use (All Water Sources) (kgal)','Largest Property Use Type - Gross Floor Area (ft²)','index','Order','Property Id']

prep_pipeline = Pipeline([
        ('deleter', MissVals_Deleter()),
        ('outremover', Outliers_Remover(['Site EUI (kBtu/ft²)'])),
        ('feature_encoder', Feature_Encoder(reg_target,['Borough', 'Largest Property Use Type'])),
        ('speccolumn_remover', SpecColumn_Remover(spec_cols)),
        ('strattraintest_splitter', StratTrainTest_Splitter(reg_target, [0., 20., 45., 70., 90., np.inf], [1, 2, 3, 4, 5], verbose = True)),
    ])

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(missing_values=np.nan, strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

X, X_test, y, y_test = prep_pipeline.fit_transform(data)

Your selected dataframe has 60 columns.
There are 43 columns that have missing values.
By the remove percentage criterion 50, we may remove 19 columns.


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


(0, 96)
(24724, 96)
               Overall  Stratified    Random  Rand. %error  Strat. %error
(0.0, 20.0]   0.196449    0.196414  0.195201     -0.635245      -0.017647
(20.0, 45.0]  0.197177    0.197223  0.199110      0.980561       0.023400
(45.0, 70.0]  0.223184    0.223106  0.220005     -1.424188      -0.034945
(70.0, 90.0]  0.198269    0.198301  0.199110      0.424365       0.016412
(90.0, inf]   0.184922    0.184956  0.186573      0.893172       0.018375


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [7]:
X = num_pipeline.fit_transform(X)
X_test = num_pipeline.fit_transform(X_test)
X.shape

(17306, 95)

In [8]:
def results_display(reg, X, X_test, y, y_test):
    from sklearn.metrics import mean_squared_error, r2_score
    from timeit import default_timer as timer
    start = timer()
    reg.fit(X, y)
    #train set
    X_pred = reg.predict(X)
    X_rmse = np.sqrt(mean_squared_error(y, X_pred))
    X_R2 = r2_score(y, X_pred)
    #test set
    X_test_pred = reg.predict(X_test)
    X_test_rmse = np.sqrt(mean_squared_error(y_test, X_test_pred))
    X_test_R2 = r2_score(y_test, X_test_pred)
    end = timer()
    print(str(reg))
    print('Train set: Regressor name, RMSE, R2')
    print(reg.__class__.__name__, round(X_rmse,3), round(X_R2,3))
    print('Test set: Regressor name, RMSE, R2')
    print(reg.__class__.__name__, round(X_test_rmse,3), round(X_test_R2,3))
    print('Time consumption [s]: ', round((end - start),3))
    print('---------------------------------------------------------')

In [9]:
from sklearn.linear_model import Ridge
base_reg=Ridge(alpha=1.0, fit_intercept=True, normalize=False, copy_X=True, max_iter=None, tol=0.001, solver='auto', random_state=None)

from sklearn.svm import SVR
base_reg2=SVR(kernel='rbf', degree=3, gamma='scale', coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1)

from sklearn.model_selection import GridSearchCV

In [10]:
linear=SVR(kernel='linear', degree=3, gamma='scale', coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1)

In [11]:
results_display(linear, X, X_test, y, y_test)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
Train set: Regressor name, RMSE, R2
SVR 17.543 0.699
Test set: Regressor name, RMSE, R2
SVR 18.052 0.679
Time consumption [s]:  360.486
---------------------------------------------------------


In [12]:
from sklearn.metrics import mean_squared_error, r2_score
from timeit import default_timer as timer

def full_analyzer(X, X_test, y, y_test):
    start = timer()

    param_grid = [
        # try 20 (5×4) combinations of hyperparameters
        {'alpha': [1.0, 0.1, 0.01, 0.001, 0.0001], 'max_iter': [200, 500, 1000, 2000]},
        ]

    ridge_reg = Ridge(fit_intercept=True, random_state=4711)
    # train across 5 folds, that's a total of 20*5=100 rounds of training 
    grid_search = GridSearchCV(ridge_reg, param_grid, cv=5,
                               scoring='neg_mean_squared_error',
                               return_train_score=True)
    grid_search.fit(X, y)
    grid_best_est=grid_search.best_estimator_
    print('Best estimator: '+str(grid_best_est))
    print('Best params: %s'%grid_search.best_params_)
    print('---------------------------------------------------------------------')
    cvres = grid_search.cv_results_
    for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
        print(np.sqrt(-mean_score), params)

    print('---------------------------------------------------------------------')
    results_display(grid_best_est, X, X_test, y, y_test)
    print('---------------------------------------------------------------------')

    param_grid = [
        # try 2 combinations of hyperparameters
        {'gamma': ['scale', 'auto']},
        ]
 
    svr_reg = SVR(kernel='rbf', degree=3, coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=10000)
    # train across 5 folds, that's a total of 2*5=10 rounds of training 
    grid_search = GridSearchCV(svr_reg, param_grid, cv=5,
                               scoring='neg_mean_squared_error',
                               return_train_score=True)
    grid_search.fit(X, y)
    end = timer()
    grid_best_est=grid_search.best_estimator_
    print('Best estimator: '+str(grid_best_est))
    print('Best params: %s'%grid_search.best_params_)
    print('---------------------------------------------------------------------')
    cvres = grid_search.cv_results_
    for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
        print(np.sqrt(-mean_score), params)

    print('---------------------------------------------------------------------')
    results_display(grid_best_est, X, X_test, y, y_test)
    print('Full time consumption [s]: ', round((end - start),3))
    print('---------------------------------------------------------------------')

In [13]:
full_analyzer(X, X_test, y, y_test)

Best estimator: Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=200, normalize=False,
      random_state=4711, solver='auto', tol=0.001)
Best params: {'alpha': 1.0, 'max_iter': 200}
---------------------------------------------------------------------
17.343897308434304 {'alpha': 1.0, 'max_iter': 200}
17.343897308434304 {'alpha': 1.0, 'max_iter': 500}
17.343897308434304 {'alpha': 1.0, 'max_iter': 1000}
17.343897308434304 {'alpha': 1.0, 'max_iter': 2000}
17.344629902747034 {'alpha': 0.1, 'max_iter': 200}
17.344629902747034 {'alpha': 0.1, 'max_iter': 500}
17.344629902747034 {'alpha': 0.1, 'max_iter': 1000}
17.344629902747034 {'alpha': 0.1, 'max_iter': 2000}
17.344839847362586 {'alpha': 0.01, 'max_iter': 200}
17.344839847362586 {'alpha': 0.01, 'max_iter': 500}
17.344839847362586 {'alpha': 0.01, 'max_iter': 1000}
17.344839847362586 {'alpha': 0.01, 'max_iter': 2000}
17.34486541022375 {'alpha': 0.001, 'max_iter': 200}
17.34486541022375 {'alpha': 0.001, 'max_iter': 500}
17.34486541

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Best estimator: SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=10000, shrinking=True, tol=0.001, verbose=False)
Best params: {'gamma': 'scale'}
---------------------------------------------------------------------
15.174804764461774 {'gamma': 'scale'}
15.513599389067934 {'gamma': 'auto'}
---------------------------------------------------------------------


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=10000, shrinking=True, tol=0.001, verbose=False)
Train set: Regressor name, RMSE, R2
SVR 14.85 0.784
Test set: Regressor name, RMSE, R2
SVR 15.253 0.771
Time consumption [s]:  373.977
---------------------------------------------------------
Full time consumption [s]:  2370.869
---------------------------------------------------------------------


observations:
- we have 95 features
- Ridge is very fast, ~0.03s per run on ThinkPad Core i7
- Ridge has R2 scores of ~71% on training set and ~69% on test set
- SVR is very slow, ~400s per run on ThinkPad Core i7
- SVR has R2 scores of ~78% on training set and ~77% on test set

### Second exercise: Ridge  and SVR rbf with the sqrt-log feature adder

In [14]:
spec_cols=['Water Use (All Water Sources) (kgal)','log_Water Use (All Water Sources) (kgal)','Largest Property Use Type - Gross Floor Area (ft²)','index','Order','Property Id']

prep_pipeline = Pipeline([
        ('deleter', MissVals_Deleter()),
        ('outremover', Outliers_Remover(['Site EUI (kBtu/ft²)'])),
        ('feature_encoder', Feature_AdderEncoder(reg_target,['Borough', 'Largest Property Use Type'])),
        ('speccolumn_remover', SpecColumn_Remover(spec_cols)),
        ('strattraintest_splitter', StratTrainTest_Splitter(reg_target, [0., 20., 45., 70., 90., np.inf], [1, 2, 3, 4, 5], verbose = True)),
    ])

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(missing_values=np.nan, strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

X, X_test, y, y_test = prep_pipeline.fit_transform(data)

Your selected dataframe has 60 columns.
There are 43 columns that have missing values.
By the remove percentage criterion 50, we may remove 19 columns.


/notebooks/ML/pipe_classes.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_subset['sqrt_' + col] = np.sqrt(numeric_subset[col])
/notebooks/ML/pipe_classes.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_subset['log_' + col] = np.log(numeric_subset[col])
/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:856: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:856:

(0, 143)
(24724, 143)
               Overall  Stratified    Random  Rand. %error  Strat. %error
(0.0, 20.0]   0.196449    0.196414  0.195201     -0.635245      -0.017647
(20.0, 45.0]  0.197177    0.197223  0.199110      0.980561       0.023400
(45.0, 70.0]  0.223184    0.223106  0.220005     -1.424188      -0.034945
(70.0, 90.0]  0.198269    0.198301  0.199110      0.424365       0.016412
(90.0, inf]   0.184922    0.184956  0.186573      0.893172       0.018375


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [15]:
X = num_pipeline.fit_transform(X)
X.shape

(17306, 142)

In [16]:
X_test = num_pipeline.fit_transform(X_test)

In [18]:
start = timer()

param_grid = [
    # try 20 (5×4) combinations of hyperparameters
    {'alpha': [1.0, 0.1, 0.01, 0.001, 0.0001], 'max_iter': [200, 500, 1000, 2000]},
    ]

ridge_reg = Ridge(fit_intercept=True, random_state=4711)
# train across 5 folds, that's a total of 20*5=100 rounds of training 
grid_search = GridSearchCV(ridge_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X, y)
grid_best_est=grid_search.best_estimator_
print('Best estimator: '+str(grid_best_est))
print('Best params: %s'%grid_search.best_params_)
print('---------------------------------------------------------------------')
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

print('---------------------------------------------------------------------')
results_display(grid_best_est, X, X_test, y, y_test)
print('---------------------------------------------------------------------')

param_grid = [
    # try 1 combination of hyperparameters ('scale' worked best)
    {'gamma': ['scale']},
    ]

svr_reg = SVR(kernel='rbf', degree=3, coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=10000)
# train across 5 folds, that's a total of 2*5=10 rounds of training 
grid_search = GridSearchCV(svr_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(X, y)
end = timer()
grid_best_est=grid_search.best_estimator_
print('Best estimator: '+str(grid_best_est))
print('Best params: %s'%grid_search.best_params_)
print('---------------------------------------------------------------------')
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

print('---------------------------------------------------------------------')
results_display(grid_best_est, X, X_test, y, y_test)
print('Full time consumption [s]: ', round((end - start),3))
print('---------------------------------------------------------------------')

Best estimator: Ridge(alpha=0.0001, copy_X=True, fit_intercept=True, max_iter=200,
      normalize=False, random_state=4711, solver='auto', tol=0.001)
Best params: {'alpha': 0.0001, 'max_iter': 200}
---------------------------------------------------------------------
16.113684468891513 {'alpha': 1.0, 'max_iter': 200}
16.113684468891513 {'alpha': 1.0, 'max_iter': 500}
16.113684468891513 {'alpha': 1.0, 'max_iter': 1000}
16.113684468891513 {'alpha': 1.0, 'max_iter': 2000}
16.113281033825793 {'alpha': 0.1, 'max_iter': 200}
16.113281033825793 {'alpha': 0.1, 'max_iter': 500}
16.113281033825793 {'alpha': 0.1, 'max_iter': 1000}
16.113281033825793 {'alpha': 0.1, 'max_iter': 2000}
16.115105050469133 {'alpha': 0.01, 'max_iter': 200}
16.115105050469133 {'alpha': 0.01, 'max_iter': 500}
16.115105050469133 {'alpha': 0.01, 'max_iter': 1000}
16.115105050469133 {'alpha': 0.01, 'max_iter': 2000}
16.114093472935753 {'alpha': 0.001, 'max_iter': 200}
16.114093472935753 {'alpha': 0.001, 'max_iter': 500}
16.

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Best estimator: SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=10000, shrinking=True, tol=0.001, verbose=False)
Best params: {'gamma': 'scale'}
---------------------------------------------------------------------
14.611134324670523 {'gamma': 'scale'}
---------------------------------------------------------------------


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=10000, shrinking=True, tol=0.001, verbose=False)
Train set: Regressor name, RMSE, R2
SVR 14.223 0.802
Test set: Regressor name, RMSE, R2
SVR 14.598 0.79
Time consumption [s]:  525.354
---------------------------------------------------------
Full time consumption [s]:  1785.489
---------------------------------------------------------------------


observations:
- we have 142 features
- Ridge is very fast, ~0.05s per run on ThinkPad Core i7
- Ridge has R2 scores of ~75% (base: 71%) on training set and ~69% (base: 69%) on test set (why is there a larger discrepancy)
- SVR is very slow, ~525s per run on ThinkPad Core i7
- SVR has R2 scores of ~80% (base: 78%) on training set and ~79% (base: 77%) on test set